In [1]:
import os
import sys

sys.path.insert(0, '../src')

from decimal import Decimal
from datetime import datetime

from the_pitch.data import Query
from the_pitch.simulators import PitchSimulator, Scorer
from the_pitch.strategies import StrategyFactory, StrategyType

In [2]:
symbols=['DAL', 'AAL', 'ALK', 'BA']

if not os.path.exists('./data/airlines.csv'):
    df = Query.get_pricing_data(source='yahoo', symbols=symbols, start='2017-01-01', end='2021-01-01')
    df.to_csv('./data/airlines.csv')

dataset = Query.load_dataset(csv = './data/airlines.csv', now='2018-06-01')

In [3]:
simulator = PitchSimulator(dataset)

In [3]:
## get strategy, indicators
strategy, indicators = StrategyFactory.create(
    id='TS1',
    symbols=symbols,
    strategy_type=StrategyType.SMA50_X_SMA200,
)

## run simulator
operations = simulator.run(
    indicators=indicators,
    strategies=[ strategy ]
)

In [4]:
scores = Scorer.score(operations)
scores

{'TS1': {'DAL': Decimal('-13.74'),
  'BA': Decimal('44.39'),
  'ALK': Decimal('-21.41')}}

In [7]:
## expected,
{
    'TS1': {
        'DAL': Decimal('-13.74'),
        'BA': Decimal('44.39'),
        'ALK': Decimal('-21.41')
    }
}

{'TS1': {'DAL': Decimal('-13.74'),
  'BA': Decimal('44.39'),
  'ALK': Decimal('-21.41')}}